

# Proyecto

### Equipo:

- Felipe Aguilar M.
- Joaquín Pérez O.

- Nombre usuario codalab: jpolar

- Nombre equipo codalab: feljo

### Link de repositorio de GitHub: `https://github.com/JoaquinPerez17/Lab-MDS7202`




## 1. Introducción

El Proyecto consiste en resolver dos problemas predictivos para el afamado *influencer* Don Rene en su aventura por el rubro del asesoramiento de inversionistas en la industria de los videojuegos. 

El primero de ellos, consiste en resolver un problema de **clasificación** que entregue la potencial evaluación que le den los usuarios que jugarán el juego. Mientras que el otro, es un problema de **regresion** de las porenciales ventas que tendrán los juegos. Para esto, Don Rene dispone de un dataset que cuenta con diversa información de los juegos de la plataforma Steam (tales como fecha de lanzamiento, logros disponibles, plataforma, desarrollador, distribuidor, tags, etc...) más las etiquetas y valores a predecir.

El dataset esta compuesto por 7881 datos, cada dato contiene distintas features que caracterizan los juegos donde se pueden encontrar variables del tipo categórica, numérico, textos y fechas, además de contar con las distintas clases que tiene cada juego y que son el objetivo a asignar para el problema de clasificación. Por último, se entregan las ventas estimadas de cada juego y corresponde al atributo a predecir en el problema de regresión.

Para el primer problema que consiste en clasificar, se evalúa en base a la métrica del f1_weighted, esta permite medir el desempeño global del modelo mediante la combinación de precision-recall, determinando una métrica para cada clase y luego haciendo un promedio ponderado por el número de ejemplos de cada clase. En el segundo problema, se evalúa en base a el Coeficiente de determinacion $R^2$, que determina la capacidad de un modelo para predecir futuros resultados. El mejor resultado posible es 1.

La metodología seguida para escoger los modelos en este proyecto, es primero probar unos modelos básicos que establezcan un límite inferior al mejor modelo que como grupo sea posible encontrar, ese modelo básico o baseline debe ser mejor que un modelo aleatorio dummy. Teniendo el modelo baseline, se probaron una serie de otros modelos, y se optimizaron únicamente los mejores modelos para cada problema a resolver. En ambos problemas los mejores modelos eran los basados en gradient boosting/bagging.

De manera sintetizada, este proyecto requiere de una serie de transformaciones intermedias a la data, a grandes rasgos, se realiza un escalamiento a las variables numéricas y una vectorización de las columnas con texto.

Se considera que los modelos propuestos logran superar los modelos baseline, obteniendo una performance aceptable en el contexto del problema. En cuanto a la comparación con los equipos, en el primer problema se alcanza un resultado promedio, muy cercano a los otros equipos, mientras que en el segundo se obtiene un excelente resultado, alcanzando el cuarto lugar de la competencia.

---
## 2. Análisis Exploratorio de Datos

In [ ]:
!pip uninstall umap
!pip install umap-learn
import umap.umap_ as umap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 3.6 MB/s 
     |████████████████████████████████| 1.1 MB 57.8 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=d4eaa590b9de14d9cf9f37dd5ca4c7331dc8f2030b8e9b4a7a7e93d205e7bdb4
  Stored in directory: /root/.cache/pip/wheels/a9/3a/67/06a8950e053725912e6a8c42c4a3a241410f6487b8402542ea
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl size=55513 sha256=6c66cd25cb2a091fc072e88091f0776b5d28fadbe42d83134ef1044fc3099d8f
  Stored in directory: /root/.cache/pip/wheels/1c/63/3a/29954bca1a27ba100ed8c27973a78cb71b43dc67aed62e80c3
Successfully built umap-learn pynndescent


En primer lugar se leen los datos de entrenamiento y test para revisar su estructura y evidenciar posibles diferencias entre sus atributos.

In [ ]:
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.pickle to test.pickle
Saving train.pickle to train.pickle


In [ ]:
obj = pd.read_pickle('/content/train.pickle')
test = pd.read_pickle('/content/test.pickle')
display(obj.head())
display(test.head())

,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,tags,achievements,average_playtime,price,short_description,estimated_sells,rating
0,An Aspie Life,2018-03-29,1,Bradley Hennessey;Joe Watson,EnderLost Studios,windows,0,Single-player;Steam Achievements,Adventure;Casual;Free to Play;Indie;Simulation,Free to Play;Adventure;Indie,23,0,0.00,One day your roommate Leaves for no reason. Yo...,3914,Mixed
1,GhostControl Inc.,2014-06-06,1,bumblebee,Application Systems Heidelberg,windows;mac;linux,0,Single-player;Steam Achievements;Steam Trading...,Casual;Indie;Simulation;Strategy,Turn-Based;Indie;Simulation,53,65,10.99,Manage a team of ghosthunters and free London ...,10728,Mixed
2,Deponia,2012-08-06,1,Daedalic Entertainment,Daedalic Entertainment,windows;mac;linux,0,Single-player;Steam Achievements;Steam Trading...,Adventure;Indie,Adventure;Point & Click;Comedy,19,217,6.99,"In Deponia, the world has degenerated into a v...",635792,Positive
3,Atlas Reactor,2016-10-04,1,Trion Worlds,Trion Worlds,windows,0,Multi-player;Online Multi-Player;Steam Achieve...,Free to Play;Strategy,Free to Play;Multiplayer;Strategy,121,1240,0.00,SEASON 6 NOW LIVE! The battle for Atlas contin...,253864,Positive
4,CHUCHEL,2018-03-07,1,Amanita Design,Amanita Design,windows;mac,0,Single-player;Steam Achievements;Steam Trading...,Adventure;Casual;Indie,Adventure;Indie;Casual,7,245,7.99,CHUCHEL is a comedy adventure game from the cr...,49818,Mostly Positive


,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,tags,achievements,average_playtime,price,short_description
0,Frog Climbers,2016-10-06,1,TeamCrew,Dear Villagers,windows,0,Single-player;Local Multi-Player;Shared/Split ...,Indie,Indie;Local Multiplayer;Funny,0,239,4.99,Climb mountains. Beat your friends. Be a douch...
1,Aztaka,2009-11-05,1,Citeremis Inc.,Citeremis Inc.,windows;mac,0,Single-player,Action;RPG;Indie,RPG;Action;Indie,0,0,3.99,"Inspired by Aztec legend, this action/rpg side..."
2,Assault Spy / アサルトスパイ,2018-10-02,1,Wazen,"NIS America, Inc.",windows,0,Single-player;Full controller support,Action;Indie,Action;Indie;Character Action Game,28,0,24.99,"Dash, evade, and smash your way to the truth a..."
3,Down To One,2016-01-07,1,Gadget Games,Gadget Games,windows,0,Multi-player;Steam Trading Cards;Stats,Action,Action;Survival;Open World,0,206,0.79,42 Players Start. Only 1 will survive.Down to ...
4,Stranded Deep,2015-01-23,1,Beam Team Games,Beam Team Pty Ltd,windows;mac;linux,0,Single-player;Full controller support;Includes...,Adventure;Indie;Early Access,Early Access;Survival;Open World,8,272,10.99,Take the role of a plane crash survivor strand...


No existe una descripción detallada de cada atributo dentro del dataset, lo que implica tomar ciertos supuestos sobre estos. En la siguiente tabla se presentan los atributos con una pequeña descripción de estos basado en los datos y el significado de cada uno y el tipo de información que define Pandas (`dtype`) 

| Columna | Descripción | dtype |
| --- | --- | --- |
| name | Nombre del juego| object |
| release_date | Fecha de lanzamiento en formato yyyy-mm-dd | object |
| english | Idioma del juego donde 1 indica que es en inglés y 0 que es otro idioma | int64 |
| developer | Desarrollador del juego | object |
| publisher | Publisher del juego | object |
| platforms | Sistema operativo compatible con el juego | object |
| required_age | Edad requerida para el usuario | int64 |
| categories | Categoría del juego en steam| object |
| genres | Genero del juego en Steam | object |
| tags | Etiquetas del juego definidas por usuarios | object |
| achievements | Cantidad de logras en el juego | int64 |
| average_playtime | Tiempo promedio del juego | int64 |
| price | Precio del juego en dólares | float64 |
| short_description | Descripción breve del juego | object |
| estimated_sells | Ventas estimadas en unidades | int64 |
| rating | Evaluación de los jugadores | category |

La mayoría de las columnas son categóricas del tipo *object* lo que es esperable para el tipo de información que entrega como es el nombre, las categorías, géneros y etiquetas entre otros, sin embargo, la columna `release_date` que corresponde a fechas debería ser transformada a tipo *datetime* de pandas. Otras variables del tipo *object* que llaman la atención son `categories`, `genres` y `tags`donde la información que entregan pareciera ser redundante, al revisar el sitio de **Steam** se observa que `categories` es jerárquicamente mayor a `genres` y `tags` y define el tipo de juego, la jugabilidad, temas, género, etc. Además, `tags` es un atributo que es definido por los usuarios a partir de etiquetas definidas por géneros y categorías y aunque existan etiquetas que no estén comprendidas en los otros atributos estas tienen la misma finalidad, por lo tanto, debido a redundancia en la información entre estos atributos es una acción a considerar unir estas dos o tres columnas para generar una sola como un atributo categórico. Dos atributos similares en su concepto son el de `developer` y `Publisher`, y se da cuenta que existen juegos donde son más de uno los que participaron, pero al no ser información redundante no se considera correcto unificarlos. Por último, se tiene `platform` y `short_description` donde la primera indica el tipo de sistema operativo compatible con el juego y la segunda una breve descripción del juego respecto a la historia y el tipo de juego.

Las columnas del tipo numérico son todas del tipo *int64* a excepción de la columna `Price`. En general no se aprecian datos redundantes o que requieran de alguna modificación, no obstante, la columna `english` que se refiere a si el juego está en idioma ingles podría tener alguna implicancia sobre el modelo dado que esta es binaria y no identifica el idioma en el caso de que este no esté en inglés.

Finalmente, las columnas `estimated_sells` y `rating` que corresponden a los objetivos señalados por Don Rene son del tipo numérica y ordinal respectivamente.



### Perfilamiento univariado

El punto inicial para comprender la naturaleza de una variable, pasa por sumarizar sus valores usando estadísticas de resumen, caracterizar su distribución y (en el caso que aplique) analizar su relación con respecto a la variable objetivo. 

El objetivo final de esta etapa es comprender qué papel juega cada variable (de forma independiente) en la resolución del problema.

#### Distribuciones de Variables Numéricas

A continuación, se presentan los histogramas para las variables numéricas que nos permiten entender y visualizar la geometría de cada distribución.

In [ ]:
num_cols = [
    "release_date",
    "required_age",
    "achievements",
    "average_playtime",
    "price",
    "estimated_sells",   
]

In [ ]:
print(f'La fecha menor es {pd.to_datetime(obj["release_date"]).min()} y la mayor {pd.to_datetime(obj["release_date"]).max()}')
display(obj[num_cols].describe())

La fecha menor es 1997-06-30 00:00:00 y la mayor 2019-05-01 00:00:00


,required_age,achievements,average_playtime,price,estimated_sells
count,7881.00000,7881.000000,7881.000000,7881.000000,7.881000e+03
mean,0.78924,43.170156,439.296790,8.431342,2.105767e+05
std,3.55538,265.399206,3303.162083,8.755668,1.513926e+06
min,0.00000,0.000000,0.000000,0.000000,3.600000e+03
25%,0.00000,0.000000,0.000000,1.990000,9.724000e+03
50%,0.00000,15.000000,27.000000,6.990000,2.150800e+04
75%,0.00000,35.000000,251.000000,11.390000,7.357300e+04
max,18.00000,9821.000000,190625.000000,78.990000,7.944129e+07


In [ ]:
fig = px.histogram(obj, x= "release_date", color="rating", marginal='box')
fig.show()

La feature `release_date` se extiende entre el 30-06-1997 y el 01-05-2019, en su distribución se muestra que la mayor cantidad de lanzamientos ocurre entre marzo y abril de 2016. Al ser la fecha una variable cíclica se pueden observar distintos aumentos en los lanzamientos y ocurren regularmente en septiembre de cada año por lo que se podría pensar que en este mes ocurren los lanzamientos de juegos más esperados, pero se debería cruzar esta información con otros atributos para validar dicha hipótesis. De todas formas, se debe incorporar la fecha dentro del dataset realizando un preprocesamiento de esta mediante aplicación de funciones trigonométricas y/o adición de estaciones del año u otra categoría de análisis anual.

In [ ]:
fig = px.histogram(obj, x="required_age", marginal='box')
fig.show()

La feature `required_age` está discretizada en 6 valores que van desde los 0 años que se entiende que son juegos para todo público y que concentran el 95.19% de los datos, luego se definen edades de 3, 7, 12, 16 y 18 años donde estos últimos toman la segunda mayoría llegando a tener 335 datos para estos grupos.

In [ ]:
fig = px.histogram(obj, x="achievements", marginal='box')
fig.show()

Los `achievements` muestran una distribución inclinada hacia la izquierda teniendo la mayor cantidad de datos en logros bajos por juego. El 75% de los datos tienen logros menores a 35 para luego distribuirse en una cantidad no menor de juegos con más logros, donde el máximo alcanza los 9821 logros seguido de otro juego que tiene 5394, esta gran diferencia podría indicar la presencia de outlier para este atributo y es una tarea a considerar en el prepocesamiento de los datos.

In [ ]:
fig = px.histogram(obj, x="average_playtime", marginal='box')
fig.show()

El `average_playtime` tiene una distribución bastante similar a la de los `achievements`, donde la mayoría de los datos se concentran en el rango de 0 a 199 horas (supuesto de la unidad de tiempo) con un total de 5361 juegos que corresponden al 68.02% de los datos. El 75% de los datos alcanza un tiempo promedio de 251.25 horas, teniendo datos que alcanzan las 190625 horas de juego en promedio, seguido de un juego que tiene como promedio de juego 95245 horas correspondiente a prácticamente la mitad del juego con más promedio de juego y que podría indicar en conjunto con los logros la presencia de un outlier en el dataset.

In [ ]:
fig = px.histogram(obj, x="price", marginal='box')
fig.show()

La feature `price` muestra un comportamiento alejado de una distribución normal, se aprecia que existen cantidades cercanas a los 500 para varios rangos de precio. La mayor cantidad de juegos son los más económicos con un total de 1101 seguido de los juegos que rondan los 7 dólares (supuesto de la moneda en la que se informan los precios). Para los demás rangos se observan cantidades de todo tipo dificultando el ajuste a alguna distribución. Los juegos más caros alcanzan valores de 78.99 dólares.

In [ ]:
fig = px.histogram(obj, x="estimated_sells", marginal='box')
fig.show()

La última variable numérica corresponde a la feature `estimated_sells` y es la clase objetivo del trabajo, esta presenta una distribución similar a la de `achievement` y `average_playtime` donde la mayor cantidad de datos se concentran entre las 0 y 100 unidades de venta estimadas, llegando a las 6235 unidades que corresponden al 79.11% de los datos. El mayor éxito de ventas superó los 79 millones de unidades vendidas, siendo mayor al segundo juego más vendido que alcanzó una cantidad de 43.44 millones.

#### Análisis Multuivariable

Por medio de este análisis se pretende determinar relaciones entre las variables numéricas.

##### **Scatter Plot Matrix**

In [ ]:
fig = px.scatter_matrix(
    obj,
    height=1000,
    dimensions = num_cols,
    color="rating",
)
fig.show()

Sobre el atributo `required_age` se puede ver un comportamiento similar al del atributo anterior debido a su discretización en los datos. Se observa que los juegos para edades mayores comenzaron a lanzarse en 2007. Para las demás variables no se muestra una correlación marcada ni comportamientos importantes a destacar. Sobre las edades se evidencia que la mayoría de los juegos están para todas las edades y las edades de 16 y 18 años y es donde se presentan los mayores precios y ventas estimadas.

Los `achievement` muestran un aumento a partir del 2012 donde anterior a este año prácticamente los logros eran cercanos a cero. Se aprecia que para el tiempo promedio de juego se agrupa sobre los ejes por lo que se esperaría una correlación baja entre estos atributos, este comportamiento se repite en las ventas estimadas.

La feature `average_playtime` muestra que en su mayoría a lo largo del tiempo es menor a 50 mil horas lo que es un tiempo elevado, pero como se vio en el histograma la mayoría de los datos no superan las 200 horas. Las ventas estimadas se agrupan en valores bajos por lo que se podría pensar que los juegos con menores tiempos promedios perciben mayores ventas. En cuanto al precio se observa que se agrupan sobre el eje del precio.

El atributo `price` muestra un aumento en el tiempo. Para los demás datos no se observan comportamientos relevantes a destacar dado que tienden a agruparse sobre una variable la mayoría de los datos.

##### **Scatter Plot Matrix**

La correlación indica la fuerza y la dirección de una relación lineal entre dos variables estadísticas.

In [ ]:
corr = obj.loc[
    :,
    num_cols,
].corr()

In [ ]:
px.imshow(
    corr,
    aspect="16:9",
    title="Correlación entre Variables Numéricas",
    height=800,
    zmin=-1,
    color_continuous_midpoint=0,
    zmax=1,
    color_continuous_scale=px.colors.sequential.Viridis,
)

Tal como se observó en el Scatter plot no se muestran correlaciones marcadas entre las variables numéricas, las que tienen una mayor correlación son el precio con la edad requerida, teniendo un valor de 0.15, que es muy bajo, algo similar se presenta entre el tiempo promedio de juego con las ventas estimadas.

#### Distribuciones de Variables categoricas

A continuación, se presentan los histogramas para las variables categóricas que nos permiten entender y visualizar la geometría de cada distribución.

In [ ]:
cat_cols = [
    "name",
    "developer",
    "publisher",
    "platforms",
    "categories",
    "genres",
    "tags",
    "short_description" ,
    "rating",
] #short_description es texto libre

In [ ]:
display(obj[cat_cols].describe())

,name,developer,publisher,platforms,categories,genres,tags,short_description,rating
count,7881,7881,7881,7881,7881,7881,7881,7881,7881
unique,7881,5365,3992,5,1933,844,3981,7848,5
top,An Aspie Life,"KOEI TECMO GAMES CO., LTD.",Ubisoft,windows,Single-player,Action;Indie,Action;Adventure;Indie,Minimal physical puzzle with explosions,Positive
freq,1,32,94,4589,756,505,68,11,2031


In [ ]:
fig = px.histogram(obj, x="english", marginal='box')
fig.show()

Para la feature `english` al ser binaria solo se pueden observar cantidades de juegos en idioma en inglés y los que no están en inglés. De los 7881 datos 7769 son juegos que están en inglés y corresponden al 98.57% y tan solo 112 son juegos en otros idiomas.

In [ ]:
fig = px.histogram(obj, x=cat_cols[1], marginal='box')
fig.show()
fig = px.histogram(obj, x=cat_cols[2], marginal='box')
fig.show()

Tanto para `developer` y `publisher` se observa que no existe una distribución marcada pero se debe al ordenamiento de los datos, debido a la gran cantidad de desarrolladores y publishers que se tienen y a como estos están definidos en cada columna, se debe realizar una exploración de los datos en la etapa de preparación donde se propone separar los distintos publisher y desarrolladores.

In [ ]:
fig = px.histogram(obj, x=cat_cols[3], marginal='box')
fig.show()

La mayor cantidad de juegos usan el sistema Windows teniendo el 87.29% del total de datos, de uso exclusivo de Windows el 58.23% de los datos corresponden a esta categoría. El SO de linux es el que tiene la menor cantidad de juegos.

Para `categories`, `genres` y `tags` se aprecia algo similar al caso de los `developer` y `publisher` donde la distribución es afectada por la forma en la que se describe cada juego, en este caso importa el orden y la cantidad de características. Tal como se mencionó anteriormente se abordará como información redundante generando una solo columna que contenga estas características del juego. Sin embargo, a pesar de todo se muestran características con frecuencias altas a pesar de las combinatorias posibles de los string que la componen.

Para las `short_description`se debe hacer un tratamiento de los textos para obtener información relevante para el clasificador y la regresión, por ahora solo mencionar que existen juegos que tienen descripciones similares y puede estar asociado al tipo de juego como los puzzles que tienen mecanicas similares entre estos.

In [ ]:
fig = px.histogram(obj, x=cat_cols[8], marginal='box')
cat_array=["Negative", "Mixed", "Mostly Positive", "Positive", "Very Positive"]
fig.update_xaxes(categoryorder="array", categoryarray=cat_array)
fig.show()

Para el `rating` se observa que estos presentan cantidades de datos similares siendo la categoría *Positive* la que tiene la mayor cantidad de datos y *Very Positive* la que tiene la menor frecuencia del dataset. La distribución de los datos esta podría considerarse con un forma normal.

### Proyección en 2D de las variables numéricas mediante UMAP

In [ ]:
df_num = obj[num_cols[1::]].copy()
df_scaled = MinMaxScaler().fit_transform(df_num)
umap = umap.UMAP(random_state = 42)
df_umap_embedded = umap.fit_transform(df_scaled)

In [ ]:
df_aux = obj.copy()
df_aux["x_umap"] = df_umap_embedded[:, 0]
df_aux["y_umap"] = df_umap_embedded[:, 1]
px.scatter(df_aux, x="x_umap", y="y_umap", color='rating',height=600)

Se observa que se tienden a formar grupos entre los datos con algunos puntos que se alejan de estos grupos. En la parte central se observa la mayor cantidad de datos, en lo que respecta a los rating no se ve que los datos se separen sino más bien se agrupan de igual forma para las distintas categorías del rating.

### Datos nulos

In [ ]:
obj.isna().sum()

name                 0
release_date         0
english              0
developer            0
publisher            0
platforms            0
required_age         0
categories           0
genres               0
tags                 0
achievements         0
average_playtime     0
price                0
short_description    0
estimated_sells      0
rating               0
dtype: int64

Tal como se ve en la tabla anterior no existen datos nulos dentro del dataset.

---

## 3. Preparación de Datos

Para preparar nuestros datos, primero analizaremos la eliminación de outliers, en cuanto a NA's no es necesario dado que el dataset no cuenta con estos por lo que su eliminación e imputación no es necesaria.

In [ ]:
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer

from sklearn.cluster import DBSCAN
import numpy as np
from sklearn.ensemble import IsolationForest

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, FunctionTransformer

from sklearn.preprocessing import FunctionTransformer

from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, HistGradientBoostingRegressor
from sklearn.metrics import classification_report
from sklearn.dummy import DummyRegressor
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.base import TransformerMixin

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.feature_selection import SelectPercentile, f_classif

from datetime import datetime, timedelta
import re

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#### Outliers

Se realiza un analisis multivariado para determinar datos como ouliers, el primero corresponde a Isolation Forest y el segundo a DBScan

In [ ]:
# El parámetro eps indicará el tamaño de la esfera y por tanto, la cantidad de outliers
# Mientras menor sea el tamaño de eps, mayor la cantidad de outliers
clustering = DBSCAN(eps=0.1, min_samples=3).fit(df_scaled)

db_scan_labels = clustering.labels_

In [ ]:
isf = IsolationForest(n_estimators=20)
outliers = isf.fit_predict(df_scaled)

In [ ]:
display(px.scatter(df_aux, x="x_umap", y="y_umap", color=outliers.astype(str),title="IsolationForest" ))
display(px.scatter(df_aux, x="x_umap", y="y_umap", color=db_scan_labels == -1,title="DBScan" ))

In [ ]:
print(f'Outliers por IsolationForest={np.count_nonzero(outliers==-1)}')
print(f'Outliers por DBScan={np.count_nonzero(db_scan_labels==-1)}')

Outliers por IsolationForest=736
Outliers por DBScan=27


#### Dataframe sin outliers

In [ ]:
df_train=obj.drop(pd.concat([df_aux,pd.Series(db_scan_labels)],axis=1)[pd.concat([df_aux,pd.Series(db_scan_labels)],axis=1)[0]==-1].index)

Dada la gran cantidad de outliers obtenidos en el IsolationForest, se descarta este para la detección de outliers, eliminando los datos del dataset mediante lo obtenido con DBScan. Es importante destacar que los juegos que se presentan como outliers son juegos exitosos como el GTA V o el PUBG. Esto puede ocasionar que se le reste peso a los estudios detrás de estos títulos y puede traer problemas al momento de determinar el éxito de los juegos.

#### Tratamiento de feature de fechas `release_date`

El feature `release_date` tiene la fecha de lanzamiento de cada juego, en terminos de Machine learning es importante poder escalar los valores de las fechas. Se propone tratarlo en un formato numérico para poder establecer una distribución y una posterior eslación de los datos.

In [ ]:
def to_timestamp(df):
    return (df - datetime(1970, 1, 1)) / timedelta(seconds=1)

df_train['release_date'] = pd.to_datetime(df_train['release_date']).apply(to_timestamp)

#### Tratamiento de feature `categories`,`genres` y `tags`

Como se observo en el EDA `genres` y `tags` son redundantes en su información, por lo que se unirán y luego se eliminarán duplicados. Se propone codificar mediante un CountVectorizer o OneHotEncoder. Lo mismo para `categories`.

In [ ]:
cols = ['genres', 'tags']
df_train['GEN'] = df_train[cols].apply(lambda x: ';'.join(x.values.astype(str)), axis=1)

#### Tratamiento de feature `platforms`

Se propone codificar mediante un CountVectorizer o OneHotEncoder. La siguiente función ejemplifica y da prueba de la transformación a realizar.

In [ ]:
def ohe_platform(df):
    df_cat = df.copy()
    return df_cat["platforms"].str.get_dummies(sep = ';')
ohe_platform(obj)

,linux,mac,windows
0,0,0,1
1,1,1,1
2,1,1,1
3,0,0,1
4,0,1,1
...,...,...,...
7876,0,0,1
7877,0,0,1
7878,0,0,1
7879,0,0,1


#### Tratamiento de feature `developer` y `publisher`

Se propone separar los desarrolladores y publishers mediante un CountVectorizer o OneHotEncoder.

#### Tratamiento de feature `short_description` 

Se propone hacer un BoW .CountVectorizer(tokenizer= SplitTokenizer())

In [ ]:
stop_words = stopwords.words('english')

class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc.lower())
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

#### ColumnTransformer


A continuación se mencionan las variables numéricas a escalar por Min_Max:

In [ ]:
minmax_num = [
    "release_date",
    "required_age",
    "achievements",
    "average_playtime",
    "price"]

A continuación se mencionan las variables categóricas que se les aplicará un OneHotEncoeder o un tratamiento con CountVectorizer.

In [ ]:
ohe_cat = [
    "developer",
    "publisher",
    "platforms",
    "categories",
    "english"]

A continuación se mencionan las variables de texto a vectorizar.

In [ ]:
tok_tex = ["short_description"]

Luego de algunas pruebas del modelo baseline, se tomó la opción de consider únicamente las variable `platforms` y `GEN` dentro de las variables categóricas, pues los resultados eran mejores. Esto puede explicarse por lo siguiente; Como `developer`, `publisher` y `categories` tienen valores únicos del orden de la cantidad de datos, la información que aportan no es muy relevante, lo mismo pasa `english`, en donde la mayoría de registros pertenencen a la opción inglés y no otra. Otra razón que potencia haber tomado esta decisión, es el hecho de que el tiempo de procesamiento es mucho menor, pues no se tienen miles y miles de opciones codificadas a través de un OneHotEncoder o CountVectorizer.

In [ ]:
sep = r'\;'
vectorizer = CountVectorizer(tokenizer=lambda x: list(set(re.split(sep, x))), decode_error='ignore')
bow = CountVectorizer(tokenizer=StemmerTokenizer(), ngram_range=(1, 2), strip_accents = 'ascii', decode_error='ignore')

In [ ]:
preprocessing_clf = ColumnTransformer([
    ('OneHotEncoder', OneHotEncoder(sparse=False, handle_unknown='ignore'), ['platforms']),
    ('Vectorizer', vectorizer, 'GEN'),
    ('MinMax', MinMaxScaler(), minmax_num),
    ('BOW', bow, 'short_description'),
])

In [ ]:
preprocessing_reg = ColumnTransformer([
    ('OneHotEncoder', OneHotEncoder(sparse=False, handle_unknown='ignore'), ['platforms']),
    ('Vectorizer', vectorizer, 'GEN'),
    ('MinMax', MinMaxScaler(), minmax_num),
    ('BOW', bow, 'short_description'),
])

---

## 4. Baseline

En primer lugar se realizará el HoldOut de los datos eliminando las columnas objetivos que serían `estimated_sells` y `rating`

In [ ]:
df_train = df_train.drop(columns=['name', 'english', 'developer', 'publisher', 'categories', 'genres', 'tags'], axis=1)

In [ ]:
X_clase, Y_clase = df_train.drop(columns=['estimated_sells', 'rating'], axis=1), df_train['rating']
X_reg, Y_reg = df_train.drop(columns=['estimated_sells', 'rating'], axis=1), df_train['estimated_sells']

Luego se hace un split de los datos para cada tarea sejando un tamaño de train del 80% para ambos.

In [ ]:
# Clasificacion
X_clase_train, X_clase_test, y_clase_train, y_clase_test = train_test_split(
    X_clase, 
    Y_clase, 
    test_size=0.33, 
    random_state=42,
    shuffle=True, 
    stratify=Y_clase
)
# Regresion
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, 
    Y_reg, 
    test_size=0.33,
    random_state=42,
    shuffle=True, 
)

#### Modelo Dummy

Se trata de un modelo de clasificación al azar que da un limite inferior al Baseline.

##### Modelos para clasificación y regresión

In [ ]:
dummy_clf = DummyClassifier(strategy='stratified', random_state=42)
dummy_reg = DummyRegressor(strategy='mean')

In [ ]:
dummy_pipeline_clf = Pipeline([('preprocessing', preprocessing_clf), ('clf', dummy_clf)])
dummy_pipeline_rg = Pipeline([('preprocessing', preprocessing_reg), ('clf', dummy_reg)])

In [ ]:
dummy_pipeline_clf.fit(X_clase_train, y_clase_train)
y_pred_clf = dummy_pipeline_clf.predict(X_clase_test)
print('Resultados DummyClassifier:')
print(classification_report(y_pred_clf, y_clase_test))

Resultados DummyClassifier:
                 precision    recall  f1-score   support

          Mixed       0.23      0.22      0.23       568
Mostly Positive       0.21      0.22      0.21       532
       Negative       0.18      0.20      0.19       400
       Positive       0.28      0.27      0.27       681
  Very Positive       0.15      0.15      0.15       411

       accuracy                           0.22      2592
      macro avg       0.21      0.21      0.21      2592
   weighted avg       0.22      0.22      0.22      2592



In [ ]:
dummy_pipeline_rg.fit(X_reg_train, y_reg_train)
y_pred_reg = dummy_pipeline_rg.predict(X_reg_test)
print('Resultados DummyRegressor:')
print(r2_score(y_reg_test, y_pred_reg))

Resultados DummyRegressor:
-0.0006554671986898075


#### Modelo Baseline

Se trata de un modelo de clasificación simple que sirve como primera iteración y debe ser mejor que el modelo Dummy.

Los Baseline serán MultinomialNB y LinearRegression

##### MultinomialNB y LinearRegression

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LinearRegression

In [ ]:
clf = MultinomialNB()
pipeline_clf = Pipeline([("preprocessing", preprocessing_clf), ('clf', clf)])

In [ ]:
pipeline_clf.fit(X_clase_train, y_clase_train)
y_pred_clf = pipeline_clf.predict(X_clase_test)
print('Resultados Clasificador 1:')
print(classification_report(y_pred_clf, y_clase_test))

Resultados Clasificador 1:
                 precision    recall  f1-score   support

          Mixed       0.25      0.31      0.27       439
Mostly Positive       0.20      0.24      0.22       480
       Negative       0.04      0.40      0.07        42
       Positive       0.66      0.28      0.39      1596
  Very Positive       0.06      0.66      0.11        35

       accuracy                           0.28      2592
      macro avg       0.24      0.38      0.21      2592
   weighted avg       0.49      0.28      0.33      2592



In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(y_clase_test, y_pred_clf, average='weighted')

0.23354264936971453

In [ ]:
preprocessing_reg = ColumnTransformer([
    ('OneHotEncoder', OneHotEncoder(sparse=False, handle_unknown='ignore'), ['platforms']),
    ('Vectorizer', vectorizer, 'GEN'),
    ('MinMax', MinMaxScaler(), minmax_num)
])

In [ ]:
reg = LinearRegression()
pipeline_reg = Pipeline([("preprocessing", preprocessing_reg), ('reg', reg)])

In [ ]:
pipeline_reg.fit(X_reg_train, y_reg_train)
y_pred_reg = pipeline_reg.predict(X_reg_test)
print('Resultados Regresor 1:')
print(r2_score(y_reg_test, y_pred_reg))

Resultados Regresor 1:
0.10806699957326593


Este es el modelo perteneciente a la segunda iteración

###### RandomForestClassifier y RandomForestRegressor

In [ ]:
clf = RandomForestClassifier()
pipeline_clf = Pipeline([("preprocessing", preprocessing_clf), ('clf', clf)])

In [ ]:
pipeline_clf.fit(X_clase_train, y_clase_train)
y_pred_clf = pipeline_clf.predict(X_clase_test)
print('Resultados Clasificador 2:')
print(classification_report(y_pred_clf, y_clase_test))

Resultados Clasificador 2:
                 precision    recall  f1-score   support

          Mixed       0.24      0.34      0.28       382
Mostly Positive       0.09      0.26      0.14       194
       Negative       0.32      0.41      0.36       335
       Positive       0.72      0.30      0.43      1596
  Very Positive       0.12      0.55      0.20        85

       accuracy                           0.33      2592
      macro avg       0.30      0.37      0.28      2592
   weighted avg       0.53      0.33      0.37      2592



In [ ]:
reg = RandomForestRegressor()
pipeline_reg = Pipeline([("preprocessing", preprocessing_reg), ('reg', reg)])

In [ ]:
pipeline_reg.fit(X_reg_train, y_reg_train)
y_pred_reg = pipeline_reg.predict(X_reg_test)
print('Resultados Regresor 2:')
print(r2_score(y_reg_test, y_pred_reg))

Resultados Regresor 2:
0.23244994308731604


Estos modelos pertenecen a la tercera iteración

##### GradientBoosting y HistGradientBoosting

In [ ]:
clf = GradientBoostingClassifier()
pipeline_clf = Pipeline([("preprocessing", preprocessing_clf), ('clf', clf)])

In [ ]:
pipeline_clf.fit(X_clase_train, y_clase_train)
y_pred_clf = pipeline_clf.predict(X_clase_test)
print('Resultados Clasificador 3:')
print(classification_report(y_pred_clf, y_clase_test))

Resultados Clasificador 3:
                 precision    recall  f1-score   support

          Mixed       0.16      0.32      0.21       276
Mostly Positive       0.08      0.24      0.12       193
       Negative       0.32      0.38      0.35       351
       Positive       0.68      0.29      0.41      1556
  Very Positive       0.25      0.45      0.32       216

       accuracy                           0.32      2592
      macro avg       0.30      0.34      0.28      2592
   weighted avg       0.49      0.32      0.35      2592



In [ ]:
f1_score(y_clase_test, y_pred_clf, average='weighted')

0.28181936248387846

In [ ]:
class Transformer2Dense(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

Para poder utilizar HistGradientBoostingRegressor es necesario llevar a cabo una transformación, de tal manera de tener características en un formato denso.

In [ ]:
reg = HistGradientBoostingRegressor()
pipeline_reg = Pipeline([('preprocessing', preprocessing_reg), ('transform2dense', Transformer2Dense()), ('reg', reg)])

In [ ]:
pipeline_reg.fit(X_reg_train, y_reg_train)
y_pred_reg = pipeline_reg.predict(X_reg_test)
print('Resultados Regresor 3:')
print(r2_score(y_reg_test, y_pred_reg))

Resultados Regresor 3:
0.33815275507991316


---

## 5. Optimización del Modelo

Ahora, agregamos selección y/o reducción de atributos y probaremos una combinación de diferentes parámetros para mejorar los mejores modelos.


In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb

Siguiendo la línea de modelos basados en gradient boosting/bagging, se prueba con AdaBoostClassifier y XGBoost

In [ ]:
pipeline_clf = Pipeline(steps=[('preprocessing', preprocessing_clf),  
                           ('select', SelectPercentile(score_func=f_classif)),
                           ("clf", MultinomialNB())])

grilla_parametros = [   
       # AdaBoostClassifier + hiperparámetros    
       {
        'select__percentile': [40, 60, 80],
        'clf': [AdaBoostClassifier(random_state=42)],
        'clf__n_estimators': [50, 150],
       },

       # XGBClassifier + hiperparámetros    
       {
        'select__percentile': [40, 60, 80],
        'clf': [xgb.XGBClassifier()],
        'clf__max_depth': [5, 7, 8],
       'clf__n_estimators': [50, 100, 200],
       }
       ]

gs_1 = HalvingGridSearchCV(pipeline_clf, grilla_parametros, scoring='f1_weighted', verbose = 10)

In [ ]:
gs_1.fit(X_clase_train, y_clase_train)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 194
max_resources_: 5262
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 33
n_resources: 194
Fitting 5 folds for each of 33 candidates, totalling 165 fits
[CV 1/5; 1/33] START clf=AdaBoostClassifier(random_state=42), clf__n_estimators=50, select__percentile=40
[CV 1/5; 1/33] END clf=AdaBoostClassifier(random_state=42), clf__n_estimators=50, select__percentile=40;, score=(train=0.529, test=0.135) total time=   0.4s
[CV 2/5; 1/33] START clf=AdaBoostClassifier(random_state=42), clf__n_estimators=50, select__percentile=40
[CV 2/5; 1/33] END clf=AdaBoostClassifier(random_state=42), clf__n_estimators=50, select__percentile=40;, score=(train=0.503, test=0.141) total time=   0.4s
[CV 3/5; 1/33] START clf=AdaBoostClassifier(random_state=42), clf__n_estimators=50, select__percentile=40
[CV 3/5; 1/33] END clf=AdaBoostClassifier(random_state=42), clf__n_estimators=50, select__percentile=40;, 

HalvingGridSearchCV(estimator=Pipeline(steps=[('preprocessing',
                                               ColumnTransformer(transformers=[('OneHotEncoder',
                                                                                OneHotEncoder(handle_unknown='ignore',
                                                                                              sparse=False),
                                                                                ['platforms']),
                                                                               ('Vectorizer',
                                                                                CountVectorizer(decode_error='ignore',
                                                                                                tokenizer=<function <lambda> at 0x7f2450c78a60>),
                                                                                'GEN'),
                                                                    

In [ ]:
print(gs_1.best_score_)
print(gs_1.best_params_)

0.315378232761866
{'clf': XGBClassifier(max_depth=5, n_estimators=200), 'clf__max_depth': 5, 'clf__n_estimators': 200, 'select__percentile': 80}


In [ ]:
best_clf = gs_1.best_estimator_

In [ ]:
y_pred = best_clf.predict(X_clase_test)
print(classification_report(y_pred, y_clase_test))

                 precision    recall  f1-score   support

          Mixed       0.27      0.33      0.29       448
Mostly Positive       0.17      0.27      0.21       362
       Negative       0.37      0.42      0.39       373
       Positive       0.52      0.31      0.38      1133
  Very Positive       0.30      0.42      0.35       276

       accuracy                           0.33      2592
      macro avg       0.32      0.35      0.33      2592
   weighted avg       0.38      0.33      0.34      2592



In [ ]:
f1_score(y_clase_test, y_pred, average='weighted')

0.3234833149630276

En este caso, se puede observar que el mejor modelo resultante de la búsqueda por grilla, logra superar por una diferencia importante al baseline de clasificación.

In [ ]:
preprocessing_reg = ColumnTransformer([
    ('OneHotEncoder', OneHotEncoder(sparse=False, handle_unknown='ignore'), ['platforms']),
    ('Vectorizer', vectorizer, 'GEN'),
    ('MinMax', MinMaxScaler(), minmax_num),
])

pipeline_reg = Pipeline(steps=[
    ('preprocessing', preprocessing_reg),
    ('transform2dense', Transformer2Dense()),
    ("reg", LinearRegression()),
])


grid_params2 = [
    # RandomForestRegressor 
    {
        "reg": [RandomForestRegressor(random_state=42)],
        "reg__criterion": ["squared_error", "absolute_error", "friedman_mse"],
        "reg__min_samples_split": [2, 5]
    },
    # HistGradientBoostingRegressor
    {
        "reg": [HistGradientBoostingRegressor(random_state=42)],
        "reg__loss": ["squared_error", "absolute_error", "poisson"],
        "reg__max_iter": [100, 150, 200],
        "reg__learning_rate": [0.1, 0.2],
        "reg__max_bins": [150, 200, 250]   
    }
]

gs_2 = HalvingGridSearchCV(pipeline_reg, grid_params2, scoring ='r2', verbose = 10)

In [ ]:
gs_2.fit(X_reg_train, y_reg_train)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 194
max_resources_: 5262
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 60
n_resources: 194
Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV 1/5; 1/60] START reg=RandomForestRegressor(random_state=42), reg__criterion=squared_error, reg__min_samples_split=2
[CV 1/5; 1/60] END reg=RandomForestRegressor(random_state=42), reg__criterion=squared_error, reg__min_samples_split=2;, score=(train=0.868, test=-0.131) total time=   0.3s
[CV 2/5; 1/60] START reg=RandomForestRegressor(random_state=42), reg__criterion=squared_error, reg__min_samples_split=2
[CV 2/5; 1/60] END reg=RandomForestRegressor(random_state=42), reg__criterion=squared_error, reg__min_samples_split=2;, score=(train=0.905, test=-0.463) total time=   0.3s
[CV 3/5; 1/60] START reg=RandomForestRegressor(random_state=42), reg__criterion=squared_error, reg__min_samples_split=2
[CV 3/5; 1/60] END reg=RandomFores

HalvingGridSearchCV(estimator=Pipeline(steps=[('preprocessing',
                                               ColumnTransformer(transformers=[('OneHotEncoder',
                                                                                OneHotEncoder(handle_unknown='ignore',
                                                                                              sparse=False),
                                                                                ['platforms']),
                                                                               ('Vectorizer',
                                                                                CountVectorizer(decode_error='ignore',
                                                                                                tokenizer=<function <lambda> at 0x7f2450c78a60>),
                                                                                'GEN'),
                                                                    

In [ ]:
print(gs_2.best_score_)
print(gs_2.best_params_)

0.2863319854736154
{'reg': HistGradientBoostingRegressor(learning_rate=0.2, loss='absolute_error',
                              max_bins=200, max_iter=200, random_state=42), 'reg__learning_rate': 0.2, 'reg__loss': 'absolute_error', 'reg__max_bins': 200, 'reg__max_iter': 200}


In [ ]:
best_reg = gs_2.best_estimator_

In [ ]:
y_pred_reg = best_reg.predict(X_reg_test)
print(r2_score(y_reg_test, y_pred_reg))

0.26936243792129333


Nuevamente, se puede observar que el mejor modelo resultante de la búsqueda por grilla, logra superar por una diferencia importante al baseline de regresión.

---

## 6. Conclusiones

Algunas respuestas que podrían plantearse pueden ser:


Según todo lo realizado, se considera que el problema fue correctamente resuelto y se lograron unos resultados aceptables. Si bien es cierto el resultado del primer problema estuvo casi empatado con el baseline de la competencia, el resultado del segundo problema estuvo muy por encima del baseline de la competencia.

El rendimiento del baseline no era muy bueno, y se logró una clara optimización, superando y mejorando ampliamente los resultados iniciales a través de la utilización de una búsqueda por grilla, en conjunto con la correcta toma de desiciones a la hora escoger los modelos finales.

Estamos conformes con los resultados finales, quizás existe una cuota de disconformidad con el resultado del primer modelo, que a pesar de estar relativamente cerca a la gran parte de los equipos, es probable que realizando una búsqueda aún más exhaustiva se podría haber conseguido un resultado excelente. Otra forma de modelar el problema, podría haber sido utilizar modelos de redes neuronales profundas que logran combinar características numéricas y texto, es probable que siguiendo ese camino los resultados serían aún mejores.

Otra forma de modelar el problema, sería tratar más meticulosamente las características que se utilizaban, pues en este caso se hizo de la forma ensayo-error, vale decir, probando qué características presentaban los mejores resultados, quedándonos únicamente con esas características, mientras que las otras se desechaban. Esto por supuesto puede ocasionar un sesgo en el conjunto de entrenamiento, afectando el resultado final.

Se considera que la competencia es una buena idea para realizar una evaluación, pues pedir que se supere una cierta barrera, hace que se trabaje en función de obtener un muy buen puntaje para tener un buen lugar en la competancia. En este sentido, el proyecto ayudó a utilizar gran parte de las herramientas aprendidas en el curso, y fue posible adentrarse un poco más a cómo sería un problema más real o concreto.


---

<br>

### Anexo: Generación de Archivo Submit de la Competencia

Para subir los resultados obtenidos a la pagina de CodaLab utilice la función `generateFiles` entregada mas abajo. Esto es debido a que usted deberá generar archivos que respeten extrictamente el formato de CodaLab, de lo contario los resultados no se veran reflejados en la pagina de la competencia.

Para los resultados obtenidos en su modelo de clasificación y regresión, estos serán guardados en un archivo zip que contenga los archivos `predicctions_clf.txt` para la clasificación y `predicctions_rgr.clf` para la regresión. Los resultados, como se comento antes, deberan ser obtenidos en base al dataset `test.pickle` y en cada una de las lineas deberan presentar las predicciones realizadas.

Ejemplos de archivos:

- [ ] `predicctions_clf.txt`

        Mostly Positive
        Mostly Positive
        Negative
        Positive
        Negative
        Positive
        ...

- [ ] `predicctions_rgr.txt`

        16103.58
        16103.58
        16041.89
        9328.62
        107976.03
        194374.08
        ...



In [ ]:
cols = ['genres', 'tags']
test['GEN'] = test[cols].apply(lambda x: ';'.join(x.values.astype(str)), axis=1)
test['release_date'] = pd.to_datetime(test['release_date']).apply(to_timestamp)

In [ ]:
X_clase_train, y_clase_train = df_train.drop(columns=['estimated_sells', 'rating'], axis=1), df_train['rating']
X_reg_train, y_reg_train = df_train.drop(columns=['estimated_sells', 'rating'], axis=1), df_train['estimated_sells']

In [ ]:
final_clf = Pipeline([('preprocessing', preprocessing_clf), 
                      ('select', SelectPercentile(percentile=80)),
                      ('clf', xgb.XGBClassifier(max_depth=5, n_estimators=200, random_state=42))])

In [ ]:
final_clf.fit(X_clase_train, y_clase_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['platforms']),
                                                 ('Vectorizer',
                                                  CountVectorizer(decode_error='ignore',
                                                                  tokenizer=<function <lambda> at 0x7f2450c78a60>),
                                                  'GEN'),
                                                 ('MinMax', MinMaxScaler(),
                                                  ['release_date',
                                                   'required_age',
                                                   'achievements',
                                                   'average_p

In [ ]:
final_reg = Pipeline([('preprocessing', preprocessing_reg),
                      ('transform2dense', Transformer2Dense()),
                      ('reg', HistGradientBoostingRegressor(learning_rate=0.2,
                                               loss='absolute_error',
                                               max_bins=200, max_iter=200,
                                               random_state=42))])

In [ ]:
final_reg.fit(X_reg_train, y_reg_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['platforms']),
                                                 ('Vectorizer',
                                                  CountVectorizer(decode_error='ignore',
                                                                  tokenizer=<function <lambda> at 0x7f2450c78a60>),
                                                  'GEN'),
                                                 ('MinMax', MinMaxScaler(),
                                                  ['release_date',
                                                   'required_age',
                                                   'achievements',
                                                   'average_p

In [ ]:
from zipfile import ZipFile
import os

def generateFiles(predict_data, clf_pipe, rgr_pipe):
    """Genera los archivos a subir en CodaLab

    Input
    predict_data: Dataframe con los datos de entrada a predecir
    clf_pipe: pipeline del clf
    rgr_pipe: pipeline del rgr

    Ouput
    archivo de txt
    """
    y_pred_clf = clf_pipe.predict(predict_data)
    y_pred_rgr = rgr_pipe.predict(predict_data)
    
    with open('./predictions_clf.txt', 'w') as f:
        for item in y_pred_clf:
            f.write("%s\n" % item)

    with open('./predictions_rgr.txt', 'w') as f:
        for item in y_pred_rgr:
            f.write("%s\n" % item)

    with ZipFile('predictions.zip', 'w') as zipObj2:
       zipObj2.write('predictions_rgr.txt')
       zipObj2.write('predictions_clf.txt')

    os.remove("predictions_rgr.txt")
    os.remove("predictions_clf.txt")

In [ ]:
generateFiles(test, final_clf, final_reg)